# Agents

## Initialization and Imports

### Imports
#### Install all the requirments from Requirments.txt file

In [22]:
%pip install -r requirements.txt --quiet


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
from langchain_experimental.agents import create_csv_agent

from langchain.agents.agent_types import AgentType
from gen_ai_hub.proxy.langchain.init_models import init_llm

Load the environment (Using your GenAI Hub Credentials)

In [7]:
from dotenv import load_dotenv
load_dotenv()

True

To view deployments of your model on GenAI Hub

In [8]:
from gen_ai_hub.proxy.core.proxy_clients import get_proxy_client
proxy_client = get_proxy_client('gen-ai-hub')
proxy_client.deployments

[Deployment(url='https://api.ai.prod.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/da2f6587d7312192', config_id='815f718b-aa6b-4341-a627-4c00226e849a', config_name='anthropic--claude-3-sonnet', deployment_id='da2f6587d7312192', model_name='anthropic--claude-3-sonnet', created_at=datetime.datetime(2024, 8, 7, 11, 41, 45, tzinfo=datetime.timezone.utc), additonal_parameters={'executable_id': 'aws-bedrock', 'model_version': 'latest'}, custom_prediction_suffix=None),
 Deployment(url='https://api.ai.prod.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/d84d6ab9eed266bb', config_id='efc68758-d247-4c7f-9baf-3b3bbabea7ad', config_name='text-embedding-ada-002-config', deployment_id='d84d6ab9eed266bb', model_name='text-embedding-ada-002', created_at=datetime.datetime(2024, 7, 31, 6, 5, 46, tzinfo=datetime.timezone.utc), additonal_parameters={'executable_id': 'azure-openai', 'model_version': 'latest'}, custom_prediction_suffix=None),
 Deployment(url='https://api.ai.p

Initializations for the CSV Files

In [9]:
csv_file_path = "./Supply_Optimizer_Log.csv"

## Agent Implementation

Different Agent Types:

ZERO_SHOT_REACT_DESCRIPTION = A zero shot agent that does a reasoning step before acting

REACT_DOCSTORE =This agent has access to a document store that allows it to look up relevant information to answering the question.

SELF_ASK_WITH_SEARCH =An agent that breaks down a complex question into a series of simpler questions. This agent uses a search tool to look up answers to the simpler questions
in order to answer the original complex question.

CONVERSATIONAL_REACT_DESCRIPTION = "conversational-react-description"
CHAT_ZERO_SHOT_REACT_DESCRIPTION = "chat-zero-shot-react-description"
A zero shot agent that does a reasoning step before acting.
This agent is designed to be used in conjunction


CHAT_CONVERSATIONAL_REACT_DESCRIPTION = "chat-conversational-react-description"

STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION = (
    "structured-chat-zero-shot-react-description"
)
An zero-shot react agent optimized for chat models.
This agent is capable of invoking tools that have multiple inputs.


OPENAI_FUNCTIONS = An agent optimized for using open AI functions.

OPENAI_MULTI_FUNCTIONS = "openai-multi-functions"

In [23]:
llm = init_llm('gpt-4', temperature=0.0)
agent = create_csv_agent(llm, csv_file_path, verbose=True, agent_type = AgentType.ZERO_SHOT_REACT_DESCRIPTION, return_intermediate_steps=True, allow_dangerous_code=True)

/Users/I552495/Library/Python/3.12/lib/python/site-packages/langchain_core/utils/utils.py:235: UserWarning: WARNING! top_p is not default parameter.
                top_p was transferred to model_kwargs.
                Please confirm that top_p is what you intended.
  warnings.warn(


In [25]:
agent.invoke("How many shipments were not fulfilled?")



> Entering new AgentExecutor chain...
Thought: To determine the number of shipments that were not fulfilled, I need to check the "Fulfilled amount" and "Requested amount" columns. If the "Fulfilled amount" is less than the "Requested amount", then the shipment was not fulfilled.

Action: python_repl_ast
Action Input: df['Fulfilled amount'] < df['Requested amount']KeyError: 'Fulfilled amount'It seems that the column names I used do not match the actual column names in the dataframe. I need to inspect the dataframe to identify the correct column names.

Action: python_repl_ast
Action Input: df.columnsIndex(['Mar-26', 'Period', 'Location ID', 'Reason', 'Period.1', 'Product ID',
       'Location ID.1', 'Ship-From Loc ID', 'Resource ID'],
      dtype='object')The column names do not include 'Fulfilled amount' or 'Requested amount' directly. These values are embedded within the 'Mar-26' column as part of a string. I need to extract these values from the string to compare them.

Action: pyt

{'input': 'How many shipments were not fulfilled?',
 'output': 'All 10,000 shipments were not fulfilled.',
 'intermediate_steps': [(AgentAction(tool='python_repl_ast', tool_input="df['Fulfilled amount'] < df['Requested amount']", log='Thought: To determine the number of shipments that were not fulfilled, I need to check the "Fulfilled amount" and "Requested amount" columns. If the "Fulfilled amount" is less than the "Requested amount", then the shipment was not fulfilled.\n\nAction: python_repl_ast\nAction Input: df[\'Fulfilled amount\'] < df[\'Requested amount\']'),
   "KeyError: 'Fulfilled amount'"),
  (AgentAction(tool='python_repl_ast', tool_input='df.columns', log='It seems that the column names I used do not match the actual column names in the dataframe. I need to inspect the dataframe to identify the correct column names.\n\nAction: python_repl_ast\nAction Input: df.columns'),
   Index(['Mar-26', 'Period', 'Location ID', 'Reason', 'Period.1', 'Product ID',
          'Location I

In [14]:
agent.invoke("On 21st May 2024, How many shipments could not be fulfilled?")



> Entering new AgentExecutor chain...
Thought: The question asks for the number of shipments that could not be fulfilled on a specific date, 21st May 2024. I need to check if there are any records for this date in the dataframe.

Action: python_repl_ast
Action Input: df['Period'].unique()['03/25/2024' '05/21/2024' '05/23/2024' '05/26/2024' '05/30/2024'
 '03/26/2024' '03/27/2024' '03/28/2024' '03/29/2024' '03/30/2024'
 '03/31/2024' '04-01-2024' '04-02-2024' '04-03-2024' '04-04-2024'
 '04-05-2024' '04-06-2024' '04-07-2024' '04-08-2024' '04-09-2024'
 '04-10-2024' '04-11-2024' '04-12-2024' '04/13/2024' '04/14/2024'
 '04/15/2024' '04/16/2024' '04/17/2024' '04/18/2024' '04/19/2024'
 '04/20/2024' '04/21/2024' '04/23/2024' '04/24/2024' '04/25/2024'
 '04/26/2024' '04/27/2024' '04/28/2024' '04/30/2024' '05-01-2024'
 '05-02-2024' '05-03-2024' '05-04-2024' '05-05-2024' '05-06-2024'
 '05-07-2024' '05-08-2024' '05-09-2024' '05-11-2024' '05-12-2024'
 '05/13/2024' '05/14/2024' '05/15/2024' '05/16/20

{'input': 'On 21st May 2024, How many shipments could not be fulfilled?',
 'output': 'On 21st May 2024, 417 shipments could not be fulfilled.'}

In [16]:
agent.invoke("On 21st May 2024, How many shipments could not be fulfilled for each product?")



> Entering new AgentExecutor chain...
Thought: To answer the question, I need to filter the dataframe `df` for the date "21st May 2024" and then count the number of unfulfilled shipments for each product. However, the provided data does not include this specific date. I should verify the dates available in the dataframe first.

Action: python_repl_ast
Action Input: df['Period'].unique()['03/25/2024' '05/21/2024' '05/23/2024' '05/26/2024' '05/30/2024'
 '03/26/2024' '03/27/2024' '03/28/2024' '03/29/2024' '03/30/2024'
 '03/31/2024' '04-01-2024' '04-02-2024' '04-03-2024' '04-04-2024'
 '04-05-2024' '04-06-2024' '04-07-2024' '04-08-2024' '04-09-2024'
 '04-10-2024' '04-11-2024' '04-12-2024' '04/13/2024' '04/14/2024'
 '04/15/2024' '04/16/2024' '04/17/2024' '04/18/2024' '04/19/2024'
 '04/20/2024' '04/21/2024' '04/23/2024' '04/24/2024' '04/25/2024'
 '04/26/2024' '04/27/2024' '04/28/2024' '04/30/2024' '05-01-2024'
 '05-02-2024' '05-03-2024' '05-04-2024' '05-05-2024' '05-06-2024'
 '05-07-2024' '

{'input': 'On 21st May 2024, How many shipments could not be fulfilled for each product?',
 'output': 'On 21st May 2024, there were 243 unfulfilled shipments for product PPC1 and 174 unfulfilled shipments for product PPC5.'}

In [17]:
agent.invoke("what is the average demand fulfillment in percentage from the given data")



> Entering new AgentExecutor chain...
Thought: To find the average demand fulfillment percentage, I need to calculate the percentage of the fulfilled amount relative to the requested amount for each row, and then find the average of these percentages.

Action: Calculate the percentage of fulfillment for each row using the formula (Fulfilled amount / Requested amount) * 100, and then find the average of these percentages.
Action Input: df['Fulfilled amount'] / df['Requested amount'] * 100Calculate the percentage of fulfillment for each row using the formula (Fulfilled amount / Requested amount) * 100, and then find the average of these percentages. is not a valid tool, try one of [python_repl_ast].I need to extract the 'Fulfilled amount' and 'Requested amount' from the 'Mar-26' column, calculate the fulfillment percentage for each row, and then find the average of these percentages.

Action: Extract the 'Fulfilled amount' and 'Requested amount' from the 'Mar-26' column using a regular

{'input': 'what is the average demand fulfillment in percentage from the given data',
 'output': 'The average demand fulfillment percentage from the given data is approximately 46.33%.'}

## Reference

- https://python.langchain.com/v0.1/docs/modules/agents/
- https://python.langchain.com/v0.1/docs/modules/agents/concepts/
- https://python.langchain.com/v0.2/docs/integrations/toolkits/csv/
- https://python.langchain.com/v0.1/docs/modules/agents/quick_start/